In [ ]:
!pip install --upgrade google-api-python-client oauth2client simplegmail
!pip -q install txtai[pipeline] > /dev/null

In [ ]:
from google.colab import files
from simplegmail import Gmail
from simplegmail.query import construct_query
from txtai.pipeline import Summary

In [ ]:

# Upload the client_secret.json file to Colab
uploaded = files.upload()  # Use this to upload the 'client_secret.json' file

# Initialize Gmail object
gmail = Gmail()  # It will open a link for OAuth authentication


In [ ]:
def get_query_params():
    query_params = {}

    # Get 'read' status
    while True:
        read_input = input("Enter 'read' status (True/False) or press Enter to skip: ")
        if read_input == '':
            break  # Skip this field if user presses Enter
        elif read_input.lower() in ['true', 'false']:
            query_params["read"] = read_input.lower() == 'true'
            break
        else:
            print("Invalid input. Please enter 'True' or 'False'.")

    # Get 'sender'
    while True:
        sender_input = input("Enter sender email(s) (comma-separated) or press Enter to skip: ")
        if sender_input == '':
            break  # Skip this field if user presses Enter
        elif all('@' in email.strip() for email in sender_input.split(',')):
            query_params["sender"] = [email.strip() for email in sender_input.split(',')]
            break
        else:
            print("Invalid input. Please enter valid email addresses.")

    # Get 'newer_than'
    while True:
        newer_than_input = input("Enter 'newer_than' duration (e.g., '5 day') or press Enter to skip: ")
        if newer_than_input == '':
            break  # Skip this field if user presses Enter
        try:
            number, unit = newer_than_input.split()
            query_params["newer_than"] = (int(number), unit)
            break
        except ValueError:
            print("Invalid input for 'newer_than'. It should be in the format: '<number> <unit>'")

    # Get 'labels'
    while True:
        labels_input = input("Enter labels (comma-separated, e.g., 'CATEGORY_FORUMS, INBOX') or press Enter to skip: ")
        if labels_input == '':
            break  # Skip this field if user presses Enter
        elif all(label.strip() for label in labels_input.split(',')):
            query_params["labels"] = [[label.strip()] for label in labels_input.split(',')]
            break
        else:
            print("Invalid input. Please enter valid labels.")

    return query_params

# Call the function to get user input and print the resulting query_params
query_params = get_query_params()
print("\nGenerated query_params:")
print(query_params)


In [ ]:

promo_query = construct_query(query_params)

output = gmail.get_messages(query=promo_query)

In [ ]:
# Create and run pipeline
summary = Summary()

In [ ]:
mail_contents=""
# Assuming `output` is a list of mail objects
for i, mail in enumerate(output):  # Use enumerate to get an index
    mail_contents += f"Sender: {mail.sender}\n"
    mail_contents += f"Subject: {mail.subject}\n"
    mail_contents += f"Date: {mail.date}\n"
    # Writing plain text part of mail
    if mail.plain:
      mail_contents+=summary(mail.plain)
    else:
      mail_contents+="No plain text inside emails"
    mail_contents+="\n\n\n"  # Add spacing between emails
with open('mail_collector.txt', 'a') as col:
  col.write(mail_contents)
print('Write completed')


In [ ]:
# email=input("enter your email address:\n")
email = "poojapriya2705@gmail.com"
params={
    "to":email,
    "sender":"poojapriya2705@gmail.com",
    "subject":"A summary for your inbox",
    "msg_plain":mail_contents
}

# Send the email
message=gmail.send_message(**params)

print('Email sent successfully!')